## 0. Install Dependencies

In [24]:
!pip install tensorflow
!pip install gym
!pip install keras 
!pip install keras-rl2

## 1. Test Random Environment with OpernAI Gym

In [25]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [26]:
class SpeedControllerEnv(Env):
    def __init__(self):
        # Actions we can take, down the speed, stay, up the speed
        self.action_space = Discrete(3)
        # Speed array
        self.observation_space = Box(low=np.array([0]), high=np.array([120]))
        # Set start temp
        self.state = 60 + random.randint(-10,10)
        # Set acceleration length
        self.acceleration_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 speed
        # 1 -1 = 0 
        # 2 -1 = 1 speed 
        self.state += action -1 
        # Reduce acceleration length by 1 second
        self.acceleration_length -= 1 
        
        # Calculate reward
        if self.state >=50 and self.state <=70: 
            if self.state>=55 and self.state<=65:
                reward =2
            else:
                reward=-1
        else: 
            reward = -1 
        
        # Check if speed is done
        if self.acceleration_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply speed noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset speed
        self.state = 60 + random.randint(-10,10)
        # Reset acceleration time
        self.acceleration_length = 60 
        return self.state

In [27]:
env = SpeedControllerEnv()

c:\Users\Apurba P. Padhy\Documents\Python\.venv\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [28]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:99
Episode:2 Score:72
Episode:3 Score:105
Episode:4 Score:-60
Episode:5 Score:-30
Episode:6 Score:42
Episode:7 Score:12
Episode:8 Score:84
Episode:9 Score:93
Episode:10 Score:51


## 2. Create a Deep Learning Model with Keras

In [29]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [30]:
states = env.observation_space.shape
actions = env.action_space.n

In [31]:

def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [32]:
del model

In [33]:
model = build_model(states, actions)

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


## 3. Build Agent with Keras RL

In [35]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [36]:

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn


In [37]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 95s 10ms/step - reward: -0.2917
166 episodes - episode_reward: -17.837 [-60.000, 120.000] - loss: 2.097 - mae: 9.882 - mean_q: -5.772

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 95s 9ms/step - reward: 1.2287
167 episodes - episode_reward: 73.599 [-57.000, 120.000] - loss: 12.625 - mae: 22.337 - mean_q: 33.157

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 95s 10ms/step - reward: 1.4423
167 episodes - episode_reward: 86.605 [-45.000, 120.000] - loss: 21.531 - mae: 35.058 - mean_q: 52.057

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 94s 9ms/step - reward: 1.4663
166 episodes - episode_reward: 88.048 [-39.000, 120.000] - loss: 27.918 - mae: 40.820 - mean_q: 60.613

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 96s 10ms/step - reward: 1.4

In [44]:
scores= dqn.test(env, nb_episodes=20, visualize=False)

Testing for 20 episodes ...
Episode 1: reward: 111.000, steps: 60
Episode 2: reward: 114.000, steps: 60
Episode 3: reward: 111.000, steps: 60
Episode 4: reward: 120.000, steps: 60
Episode 5: reward: 120.000, steps: 60
Episode 6: reward: 120.000, steps: 60
Episode 7: reward: 120.000, steps: 60
Episode 8: reward: 120.000, steps: 60
Episode 9: reward: 117.000, steps: 60
Episode 10: reward: 120.000, steps: 60
Episode 11: reward: 111.000, steps: 60
Episode 12: reward: 114.000, steps: 60
Episode 13: reward: 120.000, steps: 60
Episode 14: reward: 120.000, steps: 60
Episode 15: reward: 120.000, steps: 60
Episode 16: reward: 108.000, steps: 60
Episode 17: reward: 117.000, steps: 60
Episode 18: reward: 114.000, steps: 60
Episode 19: reward: 120.000, steps: 60
Episode 20: reward: 120.000, steps: 60


## 4. Saving model Weights to memory

In [42]:
dqn.save_weights('SpeedController_weights.h5f', overwrite=True)